In [1]:
import numpy as np
import rasterio
from rasterio.windows import Window
import rioxarray

%run _constants.ipynb



def create_ndvi_old(red_path, nir_path, dst_path):

    red_src = rasterio.open(red_path)
    nir_src = rasterio.open(nir_path)

    meta = nir_src.meta.copy()
    meta['dtype'] = 'float32'
    meta['nodata'] = NODATA_FLOAT32
    width, height = nir_src.width, nir_src.height

    with rasterio.open(dst_path, "w", **meta) as ndvi_dst:
        for row in range(height):
            red = red_src.read(1, masked=True, window=Window(0, row, width, 1))
            nir = nir_src.read(1, masked=True, window=Window(0, row, width, 1))

            # (NIR-Red) / (NIR+Red)
            ndvi = (nir.astype(np.float32) - red.astype(np.float32)) / (nir + red)

            ndvi[ndvi.mask] = 0 
            ndvi_dst.write(ndvi, window=Window(0, row, width, 1), indexes=1)
            

In [3]:
import numpy as np
import rasterio
from rasterio.windows import Window
import rioxarray

%run _constants.ipynb


def create_ndvi(red_path, nir_path, dst_path):
    
    # TODO: can just use masked?
    red_da = rioxarray.open_rasterio(red_path, chunks=(1, 1000, 1000), mask_and_scale=True)
    nir_da = rioxarray.open_rasterio(nir_path, chunks=(1, 1000, 1000), mask_and_scale=True)

    ndvi_da = (nir_da.astype(np.float32) - red_da.astype(np.float32)) / (nir_da + red_da)
    
    ndvi_da.rio.to_raster(dst_path, dtype='float32', nodata=0)
